In [15]:
import pandas as pd
from tqdm import tqdm
import utils
from copairs.map import run_pipeline
import numpy as np
import logging
import utils

logging.getLogger("copairs").setLevel(logging.INFO)

In [16]:
# Read the data
data = pd.read_parquet('../compare_parquet/data.parquet')

In [20]:
results = pd.DataFrame()

for cell in data.Metadata_Cell_type.unique():
    cell_df = data.query("Metadata_Cell_type == @cell")
    for perturbation in cell_df.Metadata_Perturbation.unique():
        perturbation_df = cell_df.query("Metadata_Perturbation == @perturbation")
        for time in perturbation_df.Metadata_Time.unique():
            time_df = (
                perturbation_df.query("Metadata_Time == @time")
                .copy()
                .assign(
                    Metadata_negcon=lambda x: x.apply(
                        lambda y: 1 if y["Metadata_control_type"] == "negcon" else 0,
                        axis=1,
                    )
                )
                .assign(
                    Metadata_Plate_Well=lambda x: x.Metadata_Plate
                    + "_"
                    + x.Metadata_Well
                )
            )

            if perturbation == "compound":
                time_df["Metadata_broad_sample"].fillna("DMSO", inplace=True)

            time_df = utils.remove_empty_wells(time_df).reset_index(drop=True)

            pos_sameby = "Metadata_broad_sample"
            pos_diffby = "Metadata_Plate_Well"
            neg_sameby = "Metadata_Plate"
            neg_diffby = "Metadata_negcon"
            null_size = 10000

            meta = utils.get_metadata(time_df)
            feats = utils.get_featuredata(time_df)

            print(f'Computing metric for {perturbation}-{cell}-{time}')

            metric = run_pipeline(
                meta, feats, pos_sameby, pos_diffby, neg_sameby, neg_diffby, null_size
            )

            results = pd.concat(
                [
                    results,
                    (
                        metric.ap.assign(Metadata_Perturbation=perturbation).assign(
                            Cell_Time=f"{cell}_{utils.time_point(perturbation, time)}"
                        )
                    ),
                ],
                axis=0,
            )

INFO:2023-04-24 16:25:23,016:copairs:Finding positive and negative pairs...
INFO:2023-04-24 16:25:23,103:copairs:Computing positive similarities...


  0%|          | 0/2 [00:00<?, ?it/s]

KeyError: "None of [Int64Index([  0,   0,   0, 768, 768, 384, 512, 512, 512, 512,\n            ...\n            703, 703, 703, 703, 703, 703, 703, 703, 703, 703],\n           dtype='int64', length=20000)] are in the [columns]"

In [ ]:
results.head()